## 1. Importação

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
%matplotlib inline

from typing import List, Dict, Tuple

path_to_dir = '../../raw/Presencial2025_dados_raw/'

df_cursos = pd.read_csv(f'{path_to_dir}dados_cursos(fixed)2025.csv')
df_disciplinas = pd.read_csv(f'{path_to_dir}dados_presencial_disciplinas2025.csv')
df_presencial_pesquisa = pd.read_csv(f'{path_to_dir}/dados_presencial_pesquisa2025.csv')
df_info_questionario = pd.read_csv(f'{path_to_dir}info_questionario_presencial2025.csv')
df_info_perguntas = pd.read_csv(f'{path_to_dir}info_perguntas_presencial2025.csv')
df_info_dimensoes = pd.read_csv(f'{path_to_dir}info_dimensoes_presencial2025.csv')

## 2. Entendendo cada DataFrame

In [2]:
def dist_variavel(df,column,savefig=True,path_to_dir='./temp_images_presencial/',title='teste'):
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=f'{column}', order=df[f'{column}'].value_counts().index)
    plt.xticks(rotation=45)
    plt.title(f'Distribuição de {column}')
    plt.ylabel('Contagem')
    plt.xlabel(f'{column}')
    plt.tight_layout()
    if savefig:
        plt.savefig(f'{path_to_dir}{title}')
    plt.close()

In [3]:
def drop_dups(df, df_name):
    print(f'Shape of {df_name} before drop_duplicates: {df.shape}')
    
    df_temp = df.drop_duplicates()
    print(f'Shape of {df_name} after drop_duplicates: {df_temp.shape}')
    
    diff = df.shape[0] - df_temp.shape[0]
    print(f'>> {diff} linhas duplicadas removidas.')
    
    return df_temp

In [4]:
def fix_split_CURSO(list):
    # print(list)
    if len(list) < 4:
        return list
    
    new_element = f'{list[0]}/{list[1]}'
    return [new_element] + list[2:]

In [5]:
def separate_column_by(df, char, column,fix_func,return_columns='TEMP'):
    df_temp = df.copy()
    
    df_temp['TEMP'] = df_temp[column].str.strip().str.split(char)
    
    # display(df_temp[['TEMP']])
    
    df_temp['TEMP'] = df_temp['TEMP'].apply(
        lambda x: [item.strip() for item in x] if isinstance(x, list) else x
    )

    df_temp['SIZE'] = df_temp['TEMP'].str.len()

    indices_problematicos = df_temp[df_temp['SIZE'] > 3].index.tolist()
    
    
    if fix_func:
        df_temp['TEMP'] = df_temp['TEMP'].apply(fix_func)
        df_temp['SIZE'] = df_temp['TEMP'].str.len()
    

    for i in range(int(df_temp['SIZE'].max())):
        df_temp[f'{return_columns}_{i}'] = df_temp['TEMP'].str[i]

    df_temp = df_temp.drop(columns=['SIZE'])

    return df_temp,indices_problematicos

### 2.1 `df_presencial_cursos`

* `COD_CURSO`
* `CURSO` : Necessita de split (CURSO,PERIODO,LOCALIDADE)
* `SETOR_CURSO`

In [6]:
df_cursos,index_problema_cursos = separate_column_by(df_cursos,char='-',fix_func=fix_split_CURSO,column='CURSO')

In [7]:
# Index arrumados manualmente
print('Index que foram consertados: ',index_problema_cursos)
df_cursos.head()

Index que foram consertados:  []


,COD_CURSO,CURSO,SETOR_CURSO,TEMP,TEMP_0,TEMP_1,TEMP_2
0,40001016027G0,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",ENGENHARIA AMBIENTAL E SANITÁRIA,Presencial,Pontal do Paraná
1,40001016047G0,FILOSOFIA - Presencial - Curitiba,SETOR DE CIÊNCIAS HUMANAS,"[FILOSOFIA, Presencial, Curitiba]",FILOSOFIA,Presencial,Curitiba
2,40001016030G0,ENGENHARIA CIVIL - Presencial - Curitiba,SETOR DE TECNOLOGIA,"[ENGENHARIA CIVIL, Presencial, Curitiba]",ENGENHARIA CIVIL,Presencial,Curitiba
3,40001016005G0,ARQUITETURA E URBANISMO - Presencial - Curitiba,SETOR DE TECNOLOGIA,"[ARQUITETURA E URBANISMO, Presencial, Curitiba]",ARQUITETURA E URBANISMO,Presencial,Curitiba
4,40001016076G0,MEDICINA VETERINÁRIA - Presencial - Curitiba,SETOR DE CIÊNCIAS AGRÁRIAS,"[MEDICINA VETERINÁRIA, Presencial, Curitiba]",MEDICINA VETERINÁRIA,Presencial,Curitiba


In [8]:
if 'MODALIDADE' not in df_cursos.columns:
    df_cursos = df_cursos.rename(columns={'TEMP_0':'CURSO','TEMP_1':'MODALIDADE','TEMP_2':'LOCAL'})

# Existem problemas a serem arrumados, portanto iremos fazer o drop no futuro
# df_cursos.drop(columns=['TEMP'], inplace=True) 

df_cursos.head()

,COD_CURSO,CURSO,SETOR_CURSO,TEMP,CURSO,MODALIDADE,LOCAL
0,40001016027G0,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,CENTRO DE ESTUDOS DO MAR,"[ENGENHARIA AMBIENTAL E SANITÁRIA, Presencial,...",ENGENHARIA AMBIENTAL E SANITÁRIA,Presencial,Pontal do Paraná
1,40001016047G0,FILOSOFIA - Presencial - Curitiba,SETOR DE CIÊNCIAS HUMANAS,"[FILOSOFIA, Presencial, Curitiba]",FILOSOFIA,Presencial,Curitiba
2,40001016030G0,ENGENHARIA CIVIL - Presencial - Curitiba,SETOR DE TECNOLOGIA,"[ENGENHARIA CIVIL, Presencial, Curitiba]",ENGENHARIA CIVIL,Presencial,Curitiba
3,40001016005G0,ARQUITETURA E URBANISMO - Presencial - Curitiba,SETOR DE TECNOLOGIA,"[ARQUITETURA E URBANISMO, Presencial, Curitiba]",ARQUITETURA E URBANISMO,Presencial,Curitiba
4,40001016076G0,MEDICINA VETERINÁRIA - Presencial - Curitiba,SETOR DE CIÊNCIAS AGRÁRIAS,"[MEDICINA VETERINÁRIA, Presencial, Curitiba]",MEDICINA VETERINÁRIA,Presencial,Curitiba


In [9]:
columns = ['MODALIDADE','LOCAL']
for i in columns:
    dist_variavel(df_cursos,i,title=f'/Distribuicao_de_{i}_before')

In [10]:
indices = df_cursos[df_cursos['TEMP'].str.len() > 3].index
print('Indíces a serem arrumados atualmente: ',indices)
# Indíces a serem arrumados manualmente:  Index([37, 63, 68, 71, 83, 95], dtype='int64')

# Índices 71,83,95 - Apagar o hífen e juntar os dois primeiros elementos com '/'
# Índices 37,68 - Apagar 'PARFOR' 
# Índices 63 - Apagar 'Técnico'

df_temp = df_cursos.loc[indices,:]

display(df_temp.loc[indices,:])

Indíces a serem arrumados atualmente:  Index([], dtype='int64')


,COD_CURSO,CURSO,SETOR_CURSO,TEMP,CURSO,MODALIDADE,LOCAL


* Dados consertados manualmente (6 observações)

Temos abaixo uma observação interessante, cuja solução será a remoção!!

In [11]:
print(df_cursos.isna().sum())
display(df_cursos[df_cursos.isna().any(axis=1)])


COD_CURSO      0
CURSO          0
SETOR_CURSO    0
TEMP           0
CURSO          0
MODALIDADE     1
LOCAL          1
dtype: int64


,COD_CURSO,CURSO,SETOR_CURSO,TEMP,CURSO,MODALIDADE,LOCAL
99,40000000000G0,PROGRAD,PRÓ-REITORIA DE GRADUAÇÃO,[PROGRAD],PROGRAD,NaN,NaN


In [12]:
# Remove a última linha (contém NaN)

mask = df_cursos.isna().any(axis=1)
index_delete = df_cursos[mask].index
print(f'Shape before drop by index: {df_cursos.shape}')
df_cursos = df_cursos.drop(index=index_delete).reset_index(drop=True)
print(f'Shape after drop by index: {df_cursos.shape}')

if len(index_delete) == 0:
    print('Tudo certo por aqui, sem mais remoções necessárias')


Shape before drop by index: (100, 7)
Shape after drop by index: (99, 7)


In [13]:
# Seleciona apenas os cursos de modalidade 'Presencial'

possible_modalidade = df_cursos['MODALIDADE'].unique().tolist()
print('Modalidades possíveis: ', possible_modalidade)
print()

print(f'Shape before selecting EAD: {df_cursos.shape}')
df_cursos = df_cursos[df_cursos['MODALIDADE'].str.contains('Presencial', case=False, na=False)]
print(f'Shape after selecting EAD: {df_cursos.shape}')

Modalidades possíveis:  ['Presencial', 'a Distância']

Shape before selecting EAD: (99, 7)
Shape after selecting EAD: (97, 7)


### 2.2 `df_presencial_disciplinas`

In [14]:
df_disciplinas,index_problema_disciplinas = separate_column_by(df_disciplinas,char='-',column='CURSO',fix_func=fix_split_CURSO)

In [15]:
print('Index problemáticos em disciplinas: ',index_problema_disciplinas)
print('Exemplo de correção: ')
display(df_disciplinas.loc[random.choice(index_problema_disciplinas),['CURSO','TEMP','TEMP_0','TEMP_1','TEMP_2']])

Index problemáticos em disciplinas:  [300, 301, 302, 303, 304, 305, 306, 307, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1131, 1132, 1133, 1134, 1135, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1553, 1554, 1555, 1599, 1600, 1739, 1740, 1741, 1742, 2014, 2015, 2016, 2017, 2065, 2091, 2092, 2093, 2464, 2592, 2593, 2594, 2595, 2693, 2694, 2695, 2696, 2799, 2800, 2801, 2802, 3430, 3431, 3432, 3433, 3434, 3589, 3590, 3591, 3592, 3593, 3676, 3677, 3678, 3679, 3680, 3681, 3682]
Exemplo de correção: 


CURSO     HISTÓRIA - MEMÓRIA E IMAGEM  - Presencial - Cu...
TEMP      [HISTÓRIA/MEMÓRIA E IMAGEM, Presencial, Curitiba]
TEMP_0                            HISTÓRIA/MEMÓRIA E IMAGEM
TEMP_1                                           Presencial
TEMP_2                                             Curitiba
Name: 2065, dtype: object

In [16]:
if 'TEMP_0' in df_disciplinas.columns:
    df_disciplinas['CURSO'] = df_disciplinas['TEMP_0']
    df_disciplinas = df_disciplinas.rename(columns={'TEMP_1':'MODALIDADE',
                                                'TEMP_2':'LOCAL'}).drop(columns=['TEMP_0']).reset_index(drop=True)

df_disciplinas.loc[index_problema_disciplinas].sample(n=1, random_state=42)

,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,MULTIPLA_ESCOLHA,CURSO,IDPROGRAMA,SETOR_CURSO,TEMP,MODALIDADE,LOCAL
858,CTP193,LÍNGUA PORTUGUESA E LITERATURA V,40001016120G0,Não,PETRÓLEO E GÁS INTEGRADO AO ENSINO MÉDIO/Técnico,40001016120G0,SETOR DE EDUCAÇÃO PROFISSIONAL E TECNOLÓGICA,[PETRÓLEO E GÁS INTEGRADO AO ENSINO MÉDIO/Técn...,Presencial,Curitiba


In [17]:
duplicados = df_disciplinas['COD_DISCIPLINA'].value_counts()

print("Top disciplinas duplicadas:")
print(duplicados[duplicados > 1].head())

# Vamos olhar um exemplo prático para entender O QUE está mudando
codigo_exemplo = duplicados[duplicados > 1].index[20]
print(f"\nExemplo de linhas para o código {codigo_exemplo}:")

# Exemplo 
display(df_disciplinas[df_disciplinas['COD_DISCIPLINA'] == codigo_exemplo])

Top disciplinas duplicadas:
COD_DISCIPLINA
CE009     21
LIB038    17
CM303     12
CM310     11
SL52      10
Name: count, dtype: int64

Exemplo de linhas para o código CI182:


,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,MULTIPLA_ESCOLHA,CURSO,IDPROGRAMA,SETOR_CURSO,TEMP,MODALIDADE,LOCAL
609,CI182,FUNDAMENTOS DE PROGRAMAÇÃO DE COMPUTADORES,40001016003G0,Não,AGRONOMIA,40001016003G0,SETOR DE CIÊNCIAS AGRÁRIAS,"[AGRONOMIA, Presencial, Curitiba]",Presencial,Curitiba
1400,CI182,FUNDAMENTOS DE PROGRAMAÇÃO DE COMPUTADORES,40001016043G0,Não,ENGENHARIA QUÍMICA,40001016043G0,SETOR DE TECNOLOGIA,"[ENGENHARIA QUÍMICA, Presencial, Curitiba]",Presencial,Curitiba
1733,CI182,FUNDAMENTOS DE PROGRAMAÇÃO DE COMPUTADORES,40001016073G0,Não,MATEMÁTICA INDUSTRIAL,40001016073G0,SETOR DE CIÊNCIAS EXATAS,"[MATEMÁTICA INDUSTRIAL, Presencial, Curitiba]",Presencial,Curitiba
1756,CI182,FUNDAMENTOS DE PROGRAMAÇÃO DE COMPUTADORES,40001016009G0,Não,BIOMEDICINA,40001016009G0,SETOR DE CIÊNCIAS BIOLÓGICAS,"[BIOMEDICINA, Presencial, Curitiba]",Presencial,Curitiba
1827,CI182,FUNDAMENTOS DE PROGRAMAÇÃO DE COMPUTADORES,40001016030G0,Não,ENGENHARIA CIVIL,40001016030G0,SETOR DE TECNOLOGIA,"[ENGENHARIA CIVIL, Presencial, Curitiba]",Presencial,Curitiba
3197,CI182,FUNDAMENTOS DE PROGRAMAÇÃO DE COMPUTADORES,40001016049G0,Não,FÍSICA,40001016049G0,SETOR DE CIÊNCIAS EXATAS,"[FÍSICA, Presencial, Curitiba]",Presencial,Curitiba


### 2.3 `df_info_questionario`

Limpeza manual pela pequena quantidade de dados.

### 2.4 `df_info_perguntas`

Informações das possíveis perguntas: Fazer merge com outras tabelas.

### 2.5 `df_info_dimensoes`

Limpeza manual pela pequena quantidade de dados.

### 2.6 `df_presencial_pesquisa`

Esse é o nosso DataFrame principal, onde trabalharemos com os merges para o processamento final

In [18]:
df_presencial_pesquisa = df_presencial_pesquisa.rename(columns={'CURSO':'TEMP'}) 
df_presencial_pesquisa.head(1)

,ID_PESQUISA,ID_QUESTIONARIO,QUESTIONARIO,ID_PERGUNTA,PERGUNTA,RESPOSTA,SITUACAO,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,MULTIPLA_ESCOLHA,TEMP,SETOR_CURSO,DEPARTAMENTO,CODPROF
0,38663,523,Disciplina: avaliação da percepção discente so...,1732,"O plano de ensino da disciplina (objetivos, co...",Discordo,Fim respostas,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,Não,ENGENHARIA AMBIENTAL E SANITÁRIA - Presencial...,CENTRO DE ESTUDOS DO MAR,Campus Pontal do Paraná - Centro de Estudos do...,PROF2259


In [19]:
df_disciplinas_unique = df_disciplinas.drop(columns=['TEMP']).drop_duplicates()

# Verificação opcional para você ver que tinha sujeira
print(f"Disciplinas antes: {df_disciplinas.shape[0]}")
print(f"Disciplinas limpas: {df_disciplinas_unique.shape[0]}")

Disciplinas antes: 3897
Disciplinas limpas: 3893


In [20]:
cols_pesquisa = df_presencial_pesquisa.columns
cols_disciplinas = df_disciplinas.columns

diff_cols_pesquisa_disciplinas = cols_disciplinas.difference(cols_pesquisa)
print(diff_cols_pesquisa_disciplinas)
print(cols_disciplinas)

Index(['CURSO', 'IDPROGRAMA', 'LOCAL', 'MODALIDADE'], dtype='object')
Index(['COD_DISCIPLINA', 'NOME_DISCIPLINA', 'COD_CURSO', 'MULTIPLA_ESCOLHA',
       'CURSO', 'IDPROGRAMA', 'SETOR_CURSO', 'TEMP', 'MODALIDADE', 'LOCAL'],
      dtype='object')


In [21]:
# "Um aluno (ID_PESQUISA) respondendo uma pergunta (ID_PERGUNTA) sobre uma matéria (NOME_DISCIPLINA) dado por um professor (CODPROF)"

colunas_unicidade = ['ID_PESQUISA', 'NOME_DISCIPLINA', 'ID_PERGUNTA','CODPROF']

print(f"Linhas antes: {df_presencial_pesquisa.shape[0]}")

df_presencial_pesquisa = df_presencial_pesquisa.drop_duplicates(subset=colunas_unicidade, keep='first')

print(f"Linhas depois: {df_presencial_pesquisa.shape[0]}")

Linhas antes: 215481
Linhas depois: 215306


In [22]:
if 'IDPROGRAMA' not in df_presencial_pesquisa.columns:
    print(f'Shape before 1° merge: {df_presencial_pesquisa.shape}')
    df_presencial_pesquisa = df_presencial_pesquisa.drop_duplicates()
    print(f'Shape after drop_duplicates: {df_presencial_pesquisa.shape}')

    colunas_add = ['NOME_DISCIPLINA','COD_CURSO','CURSO', 'IDPROGRAMA', 'LOCAL', 'MODALIDADE']
    
    # Disciplinas
    df_presencial_pesquisa = pd.merge(
        left=df_presencial_pesquisa,
        right=df_disciplinas_unique[colunas_add],
        on=['NOME_DISCIPLINA','COD_CURSO'],
        how='left'
    )

    print(f'Shape after 1° merge: {df_presencial_pesquisa.shape}')
    df_presencial_pesquisa = df_presencial_pesquisa.drop_duplicates()
    print(f'Shape after drop_duplicates: {df_presencial_pesquisa.shape}')
    df_presencial_pesquisa.head()

Shape before 1° merge: (215306, 15)
Shape after drop_duplicates: (215306, 15)
Shape after 1° merge: (228603, 19)
Shape after drop_duplicates: (215306, 19)


In [23]:
# Diferença entre as colunas de df_pesquisa para df_cursos
cols_pesquisa = df_presencial_pesquisa.columns
cols_cursos = df_cursos.columns

diff_cols_pesquisa_cursos = cols_cursos.difference(cols_pesquisa)
print(diff_cols_pesquisa_cursos)

Index([], dtype='object')


In [24]:
if len(diff_cols_pesquisa_cursos) > 0:
    print(f'Shape before 2° merge: {df_presencial_pesquisa.shape}')
    print(f'Shape before 2° merge: {df_presencial_pesquisa.shape}')
    df_presencial_pesquisa = pd.merge(
        left=df_cursos[['CURSO','SETOR_CURSO']],
        right=df_presencial_pesquisa,
        on='CURSO',
        how='left'
    )
\
    print(f'Shape after 2° merge: {df_presencial_pesquisa.shape}')
    df_presencial_pesquisa = df_presencial_pesquisa.dropna()
    print(f'Shape after dropna: {df_presencial_pesquisa.shape}')

## 3. Finalizando os Merges

In [25]:
df_merged_perguntas_dimensoes = pd.merge(
    left=df_info_perguntas.rename(columns={'Tipo_Pergunta':'Tipo_Perg'}),
    right=df_info_dimensoes[['Tipo_Perg','EIXO','DIMENSAO']],
    on='Tipo_Perg',
    how='left')

In [26]:
print(df_merged_perguntas_dimensoes.columns.difference(df_presencial_pesquisa.columns))

Index(['CL_PERGUNTA', 'DIMENSAO', 'EIXO', 'Ordem', 'Tipo_Perg'], dtype='object')


In [27]:
if 'Ordem' not in df_presencial_pesquisa.columns:
    print('Shape Before: ', df_presencial_pesquisa.shape)
    df_presencial_pesquisa = pd.merge(
        left=df_presencial_pesquisa,
        right=df_merged_perguntas_dimensoes[['ID_PERGUNTA','CL_PERGUNTA', 'DIMENSAO', 'EIXO', 'Ordem', 'Tipo_Perg']],
        on='ID_PERGUNTA',
        how='left')
    print('Shape After: ', df_presencial_pesquisa.shape)

Shape Before:  (215306, 19)
Shape After:  (215306, 24)


## 4. Identificando possíveis problemas

In [28]:
possiveis_erros = [] # Lista de Index

In [29]:
# Criação da coluna "Valor_Resposta"
df_presencial_pesquisa['VALOR_RESPOSTA'] = pd.Series([1 if i in ['Sim','Concordo'] else 0 if i == 'Desconheço' else -1 for i in df_presencial_pesquisa['RESPOSTA']])

In [30]:
sorted(df_presencial_pesquisa[['Ordem','ID_QUESTIONARIO','ID_PERGUNTA']].drop_duplicates().values.tolist(),key=lambda x: x[0]) # De todas as possíveis perguntas, somente aquelas referentes a disciplinas estão aqui. Para mais detalhes, veja: data\raw\Presencial2025_dados_raw\info_perguntas_presencial.csv

[[1, 523, 1732],
 [2, 523, 1733],
 [3, 523, 1734],
 [4, 523, 1735],
 [5, 523, 1736],
 [6, 523, 1762],
 [7, 523, 1737],
 [8, 523, 1738],
 [9, 523, 1739],
 [10, 523, 1740]]

In [31]:
df_presencial_pesquisa['DIMENSAO'].value_counts()

DIMENSAO
8    215306
Name: count, dtype: int64

In [32]:
df_presencial_pesquisa['SITUACAO'].value_counts()

SITUACAO
Fim respostas       201622
Início respostas     13684
Name: count, dtype: int64

In [33]:
comparativo = df_presencial_pesquisa.groupby(['ID_PESQUISA', 'SITUACAO']).size().unstack(fill_value=0)

display(comparativo)

print(f"\nPesquisas com contagens divergentes: {len(comparativo)}")

SITUACAO,Fim respostas,Início respostas
ID_PESQUISA,,
38663,53,0
38664,36,0
38665,80,0
38666,60,0
38667,0,50
...,...,...
46944,0,19
46947,144,0
46948,181,0



Pesquisas com contagens divergentes: 3729


In [34]:
df_presencial_pesquisa[(df_presencial_pesquisa['ID_PESQUISA']==46951) & (df_presencial_pesquisa['ID_PERGUNTA'] == 1732)]['COD_DISCIPLINA'].value_counts()

COD_DISCIPLINA
MS123    16
MM065     4
Name: count, dtype: int64

In [35]:
temp_df = df_presencial_pesquisa.groupby(['ID_PESQUISA','COD_DISCIPLINA','CODPROF'])['SITUACAO'].count()
temp_df = pd.DataFrame(temp_df)
mask = temp_df['SITUACAO']>10
temp_df[mask]

,,,SITUACAO
ID_PESQUISA,COD_DISCIPLINA,CODPROF,


In [36]:
colunas_investigacao = [
    'ID_PESQUISA', 'COD_DISCIPLINA', 'CODPROF', 'SITUACAO', 
    'COD_CURSO', 'NOME_DISCIPLINA', 'IDPROGRAMA','ID_PERGUNTA','VALOR_RESPOSTA' 
]

cod_disciplinas_investigadas = ['CM300','CQ211','CQ256','ET170']

random_disciplina = df_presencial_pesquisa['COD_DISCIPLINA'].sample(1,random_state=42).values[0]
# print(random_disciplina)

# 2. Filtra apenas esse aluno e essas disciplinas problemáticas

filtro = (df_presencial_pesquisa['ID_PESQUISA'] == 39045) & \
         (df_presencial_pesquisa['COD_DISCIPLINA'].isin(cod_disciplinas_investigadas))

df_analise = df_presencial_pesquisa.loc[filtro, colunas_investigacao]

# 3. Order
# df_analise = df_analise.sort_values(by=['ID_PESQUISA','COD_DISCIPLINA', 'CODPROF','ID_PERGUNTA'])
# df_analise

# 4. Adiciona nos possíveis erros
possiveis_erros = df_analise.index
possiveis_erros

Index([14561, 14562, 14563, 14564, 14565, 14566, 14567, 14568, 14569, 14570,
       14590, 14591, 14592, 14593, 14594, 14595, 14596, 14597, 14598, 14617,
       14618, 14619, 14620, 14621, 14622, 14623, 14624, 14625, 14626, 14627,
       14628, 14629, 14630, 14631, 14632, 14633, 14634, 14635],
      dtype='int64')

## 5. Separando os Erros dos dados Usados e exportando para .csv

In [37]:
def process(df: pd.DataFrame, axis: list[int] | None = None, dimensions: list[int] | None = None, questions: list[str] | None = None) -> pd.DataFrame:
    
        if axis is None or axis == []:
              axis = df['EIXO'].unique().tolist()
            
        if dimensions is None or dimensions == []:
            dimensions = df['DIMENSAO'].unique().tolist()

        mask_axis = df['EIXO'].isin(axis)
        mask_dim = df['DIMENSAO'].isin(dimensions)

        df_filtered = df[mask_axis & mask_dim]

        if questions is not None:
            df_filtered = df_filtered[df_filtered['PERGUNTA'].isin(questions)]

        return df_filtered


In [38]:
def separate_column_by(df, char, column,return_columns='TEMP'):
    df_temp = df.copy()
    df_temp['TEMP'] = df_temp[column].str.strip().str.split(char)
    df_temp['SIZE'] = df_temp['TEMP'].str.len()

    for i in range(df_temp['SIZE'].max()):
        df_temp[f'{return_columns}_{i}'] = df_temp['TEMP'].str[i]

    df_temp = df_temp.drop(columns=['TEMP','SIZE'])

    return df_temp

In [39]:
df_final_temp = process(df_presencial_pesquisa,axis=[],dimensions=[])
df_final_erros = df_final_temp.loc[possiveis_erros,:]
df_final = df_final_temp.drop(index=possiveis_erros)

In [40]:

# 1. Dicionário de Eixos e Dimensões com Siglas
numerical_axis_map = {
    '1': 'Planejamento e Avaliação Institucional' ,
    '2': 'Desenvolvimento Institucional' ,
    '3': 'Políticas Acadêmicas',
    '4': 'Políticas de Gestão',
    '5': 'Infraestrutura Física',
    '6': 'Complexo do Hospital de Clínicas'
}

# 2. Extrai a ordem das chaves (Eixos)
eixos = list(numerical_axis_map.keys())

def get_index_in_dict(eixo_num):
    """Retorna o índice (número) do Eixo na lista, começando por 1."""
    try:
        return numerical_axis_map[eixo_num]
    except ValueError:
        return 99 
df_final = df_final.rename(columns={'EIXO':'EIXO_NUM','DIMENSAO':'DIMENSAO_NOME'})

df_final['EIXO_NOME'] = df_final['EIXO_NUM'].astype(str).apply(lambda x: get_index_in_dict(x))

In [41]:
# Mapeamento para garantir a numeração correta
dimensao_map = {
    'PA':         8, 
    'MsãoPDI':    1,
    'RsSc':       3, 
    'PolEPPE':    2,
    'ComSoc':     4,
    'PolAD':      9, 
    'PolPes':     5,
    'OrgGesInst': 6,
    'SustFin':    10, 
    'InfraFis':   7,
    'CHC':        11
}

df_final['DIMENSAO_NUM'] = df_final['Tipo_Perg'].map(dimensao_map)

In [42]:
ordem_atual = df_final.columns
print('Ordem Atual: ',ordem_atual)
nova_ordem = ['EIXO_NUM', 'EIXO_NOME', 'DIMENSAO_NUM', 'DIMENSAO_NOME', 'ID_PESQUISA', 'ID_QUESTIONARIO', 'QUESTIONARIO', 'ID_PERGUNTA', 'PERGUNTA', 'RESPOSTA', 'VALOR_RESPOSTA', 'SITUACAO', 'COD_DISCIPLINA', 'NOME_DISCIPLINA', 'COD_CURSO', 'MULTIPLA_ESCOLHA', 'TEMP', 'SETOR_CURSO', 'DEPARTAMENTO', 'CODPROF', 'CURSO', 'IDPROGRAMA', 'LOCAL', 'MODALIDADE', 'CL_PERGUNTA',  'Ordem', 'Tipo_Perg']

if len(ordem_atual.difference(nova_ordem)) > 0:
    print('Existem colunas não assignadas. Confira abaixo:')
    print(ordem_atual.difference(nova_ordem))
    
df_final = df_final[nova_ordem]

Ordem Atual:  Index(['ID_PESQUISA', 'ID_QUESTIONARIO', 'QUESTIONARIO', 'ID_PERGUNTA',
       'PERGUNTA', 'RESPOSTA', 'SITUACAO', 'COD_DISCIPLINA', 'NOME_DISCIPLINA',
       'COD_CURSO', 'MULTIPLA_ESCOLHA', 'TEMP', 'SETOR_CURSO', 'DEPARTAMENTO',
       'CODPROF', 'CURSO', 'IDPROGRAMA', 'LOCAL', 'MODALIDADE', 'CL_PERGUNTA',
       'DIMENSAO_NOME', 'EIXO_NUM', 'Ordem', 'Tipo_Perg', 'VALOR_RESPOSTA',
       'EIXO_NOME', 'DIMENSAO_NUM'],
      dtype='object')


In [43]:
df_final.to_csv('processed_presencial_2025.csv',index=False)
df_final_erros.to_csv('processed_erros_presencial_2025.csv',index=False)

# Testes

In [44]:
df_presencial_pesquisa.groupby('ID_PESQUISA')['RESPOSTA'].count()

ID_PESQUISA
38663     53
38664     36
38665     80
38666     60
38667     50
        ... 
46944     19
46947    144
46948    181
46949    252
46951    180
Name: RESPOSTA, Length: 3729, dtype: int64

In [45]:
print(df_presencial_pesquisa['RESPOSTA'].unique())
df_presencial_pesquisa['VALOR_RESPOSTA'] = pd.Series([1 if i in ['Sim','Concordo'] else 0 if i == 'Desconheço' else -1 for i in df_presencial_pesquisa['RESPOSTA']])
df_presencial_pesquisa.head(1)

['Discordo' 'Sim' 'Concordo' 'Não' 'Desconheço']


,ID_PESQUISA,ID_QUESTIONARIO,QUESTIONARIO,ID_PERGUNTA,PERGUNTA,RESPOSTA,SITUACAO,COD_DISCIPLINA,NOME_DISCIPLINA,COD_CURSO,...,CURSO,IDPROGRAMA,LOCAL,MODALIDADE,CL_PERGUNTA,DIMENSAO,EIXO,Ordem,Tipo_Perg,VALOR_RESPOSTA
0,38663,523,Disciplina: avaliação da percepção discente so...,1732,"O plano de ensino da disciplina (objetivos, co...",Discordo,Fim respostas,EAS171,POLUIÇÃO DOS AMBIENTES COSTEIROS E MARINHOS,40001016027G0,...,ENGENHARIA AMBIENTAL E SANITÁRIA,40001016027G0,Pontal do Paraná,Presencial,Disciplina,8,1,1,AvPDSPF,-1


In [46]:
temp = df_presencial_pesquisa.groupby('ID_PESQUISA')['VALOR_RESPOSTA'].count()

In [47]:
mask1 = df_presencial_pesquisa['VALOR_RESPOSTA'] > 0
temp1 = df_presencial_pesquisa[mask1].groupby('ID_PESQUISA')['VALOR_RESPOSTA'].count()

mask2 = df_presencial_pesquisa['VALOR_RESPOSTA'] == 0
temp2 = df_presencial_pesquisa[mask2].groupby('ID_PESQUISA')['VALOR_RESPOSTA'].count()

mask3 = df_presencial_pesquisa['VALOR_RESPOSTA'] < 0
temp3 = df_presencial_pesquisa[mask3].groupby('ID_PESQUISA')['VALOR_RESPOSTA'].count()

temp_final = pd.concat([temp,temp1, temp2, temp3], axis=1, keys=['Total','Positivas', 'Desconheço', 'Negativas'])
temp_final.corr(method='spearman')

,Total,Positivas,Desconheço,Negativas
Total,1.000000,0.935002,0.316130,0.643873
Positivas,0.935002,1.000000,0.119456,0.440638
Desconheço,0.316130,0.119456,1.000000,0.149198
Negativas,0.643873,0.440638,0.149198,1.000000


In [48]:
len(df_presencial_pesquisa['ID_PESQUISA'].unique().tolist())

3729